In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Paramètres de vol 

In [2]:
Vb = 2
Diametre_bouteille = 8.9
Diametre_sortie = 0.9
Masse_fusee_vide = 0.5 # Mo
Cx = 0.1
a = 89 # prévoir de le transformer en radians pour utiliser les fonctions trigo
lr = 22
Po = 10
Veo = 0.65

## Variables à utiliser et conversions

In [3]:
# Conversion des volumes en mètres cubes
Vb /= 1000
Veo /= 1000
# Conversion de la pression en pascal
Po *= 100000 # press
# Section de la bouteille en m^2
Section_bouteille = (Diametre_bouteille**2)*np.pi/40000
Section_goulot = (Diametre_sortie**2)*np.pi/40000
lr /= 100

In [17]:
Vb

0.002

## Constantes environementales

In [4]:
g = 9.81 # gravité
r = 998 # densité de l'eau en Kg/m^3
ra = 1.2 # densité de l'air en Kg/m^3
Patm = 101325 # pression atmosphérique

## Calcul théorique

### Vitesse initiale - en sortie de rampe

In [5]:
Ax = (Po*Section_goulot-(Masse_fusee_vide+1000*Veo)*g*np.cos((90-a)*np.pi/180))/(Masse_fusee_vide+1000*Veo) # Accélération suivant x

t = np.sqrt(2*lr/Ax)

Vx = Ax*t # Vitesse au lancement

#### Supplémentaire (éventuellement à supprimer)

In [24]:
Ax_g = Ax/g # en G
Vx_kmh = Vx * 3.6

### Force de poussée - calcul du $\beta$

In [6]:
beta = r*(1 - ((Section_goulot/Section_bouteille)**2))

## Colonne 1 : Volume air en mètres cubes

In [7]:
volumes_air = [Vb - Veo]
volumes_air_final = (Po + Patm)*(Vb-Veo)/Patm
for i in range(28) :
    volumes_air.append(volumes_air[i] + (Vb-volumes_air[0])/29)

### Frontière entre les deux phases et donc ajout de deux lignes avec la valeur Vb

In [8]:
volumes_air.append(Vb)
volumes_air.append(Vb)

In [9]:
for i in range(30,48) :
    volumes_air.append(volumes_air[i] + (volumes_air_final-volumes_air[30])/19)

In [10]:
volumes_air.append(volumes_air_final)

In [13]:
len(volumes_air)

50

## Colonne 2 : Pression air en pascal

In [38]:
pression_air = list()
for i in range(len(volumes_air)) : 
    pression_air.append(((Po + Patm)*(Vb-Veo)/volumes_air[i])-Patm)

## Colonne 4 : Vitesse en m/s

### Phase 1

In [40]:
vitesse = list()
for i in range(30) :
    vitesse.append(np.sqrt(2*pression_air[i]/beta))

### Phase 2 : Formule à confirmer

In [44]:
for i in range(20) :
    vitesse.append(np.sqrt(2*pression_air[i+30]/ra))

## Colonne 3 : Temps en secondes

### Phase 1

In [23]:
temps = list()
for i in range(30) : 
    temps.append(((2/3)*volumes_air[i]**1.5 - (2/3)*(Vb-Veo)**1.5)/(Section_goulot*np.sqrt(2*Po*(Vb-Veo)/beta)))

### Phase intermédiaire

In [35]:
temps.append((((2/3)*volumes_air[30]**1.5 - (2/3)*(Vb)**1.5)/(Section_goulot*np.sqrt(2*Po*(Vb-Veo)/beta)))+temps[29])

### Phase 2

In [50]:
for i in range(19) : 
    temps.append(temps[30+i]+((volumes_air[31+i]-volumes_air[30+i])/(Section_goulot*((vitesse[31+i]+vitesse[30+i])/2))))

### Phase intermédiaire

In [55]:
temps.append(temps[49])

In [57]:
temps.append(temps[50]+0.01)

### Phase 3

In [59]:
for i in range(547) :
    temps.append(temps[i + 51]+ 0.05)

## Colonne 5 : Poussée en newton

### Phase 1

In [72]:
poussee = list()
for i in range(30) : 
    poussee.append(r*Section_goulot*vitesse[i]**2) 

### Phase 2

In [73]:
for i in range(20) :
    poussee.append(ra*Section_goulot*vitesse[30+i]**2)

## Colonne 6 : Inclinaison rampe de départ

## Colonne 7 : Masse de la fusée

### Phase 1

In [77]:
masse_fusee = list()
for i in range(30) :
    masse_fusee.append(Masse_fusee_vide+r*(Vb-volumes_air[i]))

### Phase 2

In [79]:
for i in range(569) :
    masse_fusee.append(Masse_fusee_vide)

## Colonne 8 : Vitesse de la fusée

## Colonne 9 : Résistance de l'air

## Colonne 10 : X(m)

## Colonne 11 : Y(m)

## Colonne 12 : Accélération de la fusée